In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Tokenizer,StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.sql import SQLContext
import pyspark as ps
from pyspark.ml.classification import LinearSVC
import warnings
from nltk.stem.snowball import SnowballStemmer
import pandas
import matplotlib.pyplot as plt
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
import pandas as pd


def init_spark():
    print("initializing spark...")
    try:
        sc = ps.SparkContext('local[*]')
        sqlContext = SQLContext(sc)
        print("Just created a SparkContext")
    except ValueError:
        warnings.warn("SparkContext already exists in this scope")
    spark=SparkSession.builder.getOrCreate()

def read_file(fileName):
    print("reading csv file...")
    df=spark.read.csv(fileName,sep=",",inferSchema=True,header=False)
    return df


def pre_process(df):
    print("preprocessing...")
    df.count()
    df1=df.withColumnRenamed('_c0',"id").withColumnRenamed('_c1','label').withColumnRenamed('_c2','tweet')
    df2 = df1.withColumn('tweet', regexp_replace('tweet', '[^a-z0-9A-Z`~!@#$%&<>?., ]', ''))
    df3 = df2.withColumn('tweet', regexp_replace('tweet', '[0-9`~!@#$%&<>?,\']', ''))
    df4 = df3.withColumn('tweet', regexp_replace('tweet', 'http://*.*.com', ''))
    df5 = df4.withColumn('tweet', regexp_replace('tweet', 'www.*.com', ''))
    df6 = df5.withColumn('tweet', regexp_replace('tweet', '\.', ''))
    tokenizer=Tokenizer(inputCol="tweet",outputCol="words")
    wordData=tokenizer.transform(df6)
    remover=StopWordsRemover(inputCol="words",outputCol="word_clean")
    word_clean_data=remover.transform(wordData)
    stemmer = SnowballStemmer(language='english')
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens])
    count=CountVectorizer(inputCol="word_clean",outputCol="rawFeatures")
    model=count.fit(word_clean_data)
    featurizedData=model.transform(word_clean_data)
    idf=IDF(inputCol="rawFeatures",outputCol="features")
    idfModel=idf.fit(featurizedData)
    rescaledData=idfModel.transform(featurizedData)
    return rescaledData


def train_test_split(df,train=0.7,test=0.3):
    print("splitting dataset...")
    seed=0
    trainDf,testDf=df.randomSplit([train,test],seed)
    trainDf.count()
    testDf.count()
    return trainDf,testDf

In [2]:
def SVM(train_data,test_data):
    print("................................................................................................")
    print("Using SVM model with test_data...")
    d1 = {}
    d2 = {}
    
    svm = LinearSVC(maxIter=5,regParam=0.01)
    model = svm.fit(train_data)
    
    train_pred = model.transform(train_data)
    print(train_pred.groupBy('label','prediction').count().show())
    
    my_eval_svm = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
    p1 = my_eval_svm.evaluate(train_pred)
    
    my_mc_svm = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
    p2 = my_mc_svm.evaluate(train_pred)
    
    my_mc_svm = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
    p3 = my_mc_svm.evaluate(train_pred)
    
    d1['ROC'] = p1
    d1['F1'] = p2
    d1['Accuracy'] = p3
    
    test_pred = model.transform(test_data)
    print(test_pred.groupBy('label','prediction').count().show())
    
    evaluator = BinaryClassificationEvaluator()
    evaluation = evaluator.evaluate(test_pred)
    
    my_eval_svm = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
    p4 = my_eval_svm.evaluate(test_pred)
    
    my_mc_svm = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
    p5 = my_mc_svm.evaluate(test_pred)
    
    my_mc_svm = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
    p6 = my_mc_svm.evaluate(test_pred)
    
    d2['ROC'] = p4
    d2['F1']= p5
    d2['Accuracy'] = p6
    print("................................................................................................")
    return d1,d2

In [3]:
init_spark()
df=read_file("twitter.csv")
df=pre_process(df)
train_data,test_data=train_test_split(df)

initializing spark...


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: SparkContext already exists in this scope


reading csv file...
preprocessing...
splitting dataset...


In [8]:
print("Pre-processed tweets")
df.show()

Pre-processed tweets
+---+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|label|               tweet|               words|          word_clean|         rawFeatures|            features|
+---+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  1|    1| user when a fath...|[, user, when, a,...|[, user, father, ...|(39572,[0,1,161,1...|(39572,[0,1,161,1...|
|  2|    0|user user thanks ...|[user, user, than...|[user, user, than...|(39572,[0,1,19,22...|(39572,[0,1,19,22...|
|  3|    1|user that was fuc...|[user, that, was,...|[user, fucking, w...|(39572,[1,525,126...|(39572,[1,525,126...|
|  4|    1|userthat was so s...|[userthat, was, s...|  [userthat, shitty]|(39572,[2934,3100...|(39572,[2934,3100...|
|  5|    0| factsguide socie...|[, factsguide, so...|[, factsguide, so...|(39572,[0,195,149...|(39572,[0,195,149...|
|  6|    0| huge fan fare an...|[, huge, fa

In [9]:
print(".........................................................................................")
import time
start=time.time()
train_summary,test_summary=SVM(train_data,test_data)
print(train_summary)
print(test_summary)
elapsed=time.time()-start
print("Time elapsed:",elapsed/60," mins.")
print(".........................................................................................")

.........................................................................................
................................................................................................
Using SVM model with test_data...
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|20733|
|    1|       0.0|  198|
|    1|       1.0| 1375|
|    0|       1.0|    4|
+-----+----------+-----+

None
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  370|
|    0|       0.0| 8948|
|    1|       1.0|  306|
|    0|       1.0|   28|
+-----+----------+-----+

None
................................................................................................
{'ROC': 0.9369664910967898, 'F1': 0.9906693279729695, 'Accuracy': 0.9909457642312864}
{'ROC': 0.7247716461517366, 'F1': 0.9521690921806005, 'Accuracy': 0.9587650227932035}
Time elapsed: 3.844720975557963  mins.
...........................................................